In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

%matplotlib inline

# Burger's equation


Рассмотрим 1D уравнение Бюргерса. Обычно оно представлено как:
$$
\begin{equation}
\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = 0
\end{equation}
$$

<p>
Чтобы мы могли представить схемы для более широкого спектра гиперболических
дифференциальные уравнения и для лучшей обработки ударов (т.е.
разрывы в решении), представим уравнение модели
в консервативной форме:
$$
\begin{equation}
\frac{\partial u}{\partial t} +  \frac{\partial }{\partial x} \left (\frac{u^2}{2} \right) = 0
\end{equation}
$$

и введением функции потока 
$$
\begin{equation}
 F(u) = \frac{u^2}{2}
\end{equation}
$$

<p>
консервативной формулировкой уравнения Бургерса может быть
представленное общим уравнением переноса:

$$
\begin{equation}
\frac{\partial u}{\partial t} +  \frac{\partial F(u)}{\partial x}  = 0
\end{equation}
$$
    
Методом конечного объема получаем разностную схему:
    
$$
\begin{equation}
\frac{u_j^{n+1} - u_j^{n}}{\Delta t} - \frac{\left ( F_{j-1/2} - F_{j+1/2} \right )}{\Delta x}  = 0
\end{equation}
$$
    
Аппроксимация потока задается видом:
    
$$
\begin{equation}
F_{j+\frac{1}{2}} = \frac{1}{2} [F(u_{j+1}) + F(u_{j}) - \frac{\Delta t}{\Delta x} Q(\frac{\Delta x}{\Delta t} a_{j+\frac{1}{2}}) \Delta_{j + \frac{1}{2}}u]
\end{equation}
$$
    
$$
\begin{equation}
a_{j+\frac{1}{2}} = \left\{
        \begin{array}{ll}
            \frac{\Delta_{j + \frac{1}{2}}f}{\Delta_{j + \frac{1}{2}}u}, \quad \quad \Delta_{j + \frac{1}{2}}u \neq 0 \\ 
             (\frac{\partial F}{\partial u})_j, \quad \quad \Delta_{j + \frac{1}{2}}u = 0
        \end{array}
    \right.
\end{equation}
$$
    
$$
\begin{equation}
\Delta_{j + \frac{1}{2}}u = u_{j+1} - u{j}
\end{equation}
$$
    
В зависимости от значения функции $Q(x)$ полчаются разные схемы.

## Upwind Scheme

$$
Q(x) = 0
$$

Upwind Scheme для общего уравнения сохранения имеет форму:

$$
\begin{equation}
u_{j}^{n+1} = u_{j}^{n} - \frac{\Delta t}{\Delta x} 
\left ( F(u_{j+1}^{n}) - F(u_{j}^{n}) \right )
\end{equation}
$$

где мы предположили прямую волну распространения $a(u) = F'(u)>0 $, i.e. $u>0$ для уравнения Бюргерса. С другой стороны 
$( \frac{\partial F(u)}{\partial x} )$ будет приблизительно соответствовать $\frac{1}{\Delta x} 
\left ( F(u_{j}^{n}) - F(u_{j-1}^{n}) \right )$

## Lax-Friedrich

$$
Q(x) = 1
$$

Lax-Friedrich Scheme - уравнение сохранения:

$$
\begin{equation}
u_j^{n+1} = \frac{\Delta t}{2}(u^n_{j+1}+u^n_{j-1})-\frac{F^n_{j+1}-F^n_{j-1}}{2 \Delta x}
\end{equation}
$$

## Энтропийная коррекция

Может быть задана видом:

$$
\begin{equation}
Q(x)= \left\{
        \begin{array}{ll}
            |x|, \quad \quad |x| >= \epsilon \\ 
             \frac{x^2 + \epsilon^2}{2\epsilon}, \quad \quad x < \epsilon
        \end{array}
    \right.
\end{equation}
$$

## Ссылки

1. [High Resolution Schemes for Hyperbolic Conservation Laws](https://www.sciencedirect.com/science/article/pii/0021999183901365)

2. [СРАВНЕНИЕ ТОЧНОСТИ И СХОДИМОСТИНЕКОТОРЫХ TVD-СХЕМ](https://cyberleninka.ru/article/n/sravnenie-tochnosti-i-shodimosti-nekotoryh-tvd-shem/viewer)

3. [Numerical Schemes Applied to theBurgers and Buckley-Leverett Equations](https://www.reading.ac.uk/web/files/maths/rakhib_ahmed.pdf)

4. [Numerical Methods for Hyperbolic Conservation Laws(AM257)](https://mathema.tician.de/dl/academic/notes/257/257.pdf)

In [2]:
def Q_func(x, q_mode=3, eps=0.1):
    if q_mode == 0:
        return 0
    elif q_mode == 1:
        return 1
    elif q_mode == 2:
        return entropy_fix(x, eps=eps)
    elif q_mode == 3:
        return entropy_fix(x ** 2, eps=eps)


def entropy_fix(x, eps=0.1):
    def fix(x):
        if np.abs(x) >= eps:
            return np.abs(x)
        else:
            return (x ** 2 + eps ** 2) / (2 * eps)

    fix = np.vectorize(fix)
    x = fix(x)
    return x


def a_j(u1, u2, F):
    eq_eps = 1e-5  # machine accuracy

    def fix(x1, x2):
        if np.abs(x1 - x2) < eq_eps:
            return x1
        else:
            return (F(x1) - F(x2)) / (x1 - x2)
    fix = np.vectorize(fix)
    a = fix(u1, u2)
    return a


def lax_friedrich_scheme(u, F, dt, dx, **kwargs):
    u_nj1 = u.copy()

    ujm = u[:-2].copy()  # u(j-1)
    uj = u[1:-1].copy()  # u(j)
    ujp = u[2:].copy()  # u(j+1)

    a_m = a_j(uj, ujm, F)  # a(n, j-0.5)
    a_p = a_j(ujp, uj, F)  # a(n, j+0.5)

    fjm = 0.5 * (F(uj) + F(ujm)) - 0.5 * dx / dt * Q_func(dt / dx * a_m, **kwargs)  # f(n, j-0.5)
    fjp = 0.5 * (F(uj) + F(ujp)) - 0.5 * dx / dt * Q_func(dt / dx * a_p, **kwargs)  # f(n, j+0.5)

    u_nj1[1:-1] = (u[:-2] + u[2:]) / 2.0 - (fjp - fjm) * dt / dx
    return u_nj1, u

# Lax-Wendroff на основе метода Richtmyer

Консервативная форма Лакса-Вендроффа для общего нелинейного уравнения имеет вид:

$u_i^{n+1} = u_i^n - \frac{\Delta t}{2\Delta x} \left[ f(u_{i+1}^{n}) - f(u_{i-1}^{n}) \right] + \frac{\Delta t^2}{2\Delta x^2} \left[ A_{i+1/2}\left(f(u_{i+1}^{n}) - f(u_{i}^{n})\right) - A_{i-1/2}\left( f(u_{i}^{n})-f(u_{i-1}^{n})\right) \right].$


где $A_{{i\pm 1/2}}$ is Якобиан, оцененный ${\displaystyle {\frac {1}{2}}(u_{i}^{n}+u_{i\pm 1}^{n})}$


Чтобы избежать оценки якобиана, используйте двухэтапную процедуру. Оценим поток, как

$$
\begin{equation}
F_{j+\frac{1}{2}} = F(u_{j + \frac{1}{2}}^{n + \frac{1}{2}})
\end{equation}
$$

Тогда получаем
 
$$
\begin{equation}
u_{j}^{n+1} = u_{j}^{n} - \frac{\Delta t}{\Delta x} 
\left ( F(u_{j+1/2}^{n+1/2}) - F(u_{j-1/2}^{n+1/2}) \right )
\end{equation}
$$

<p>
Первый шаг:
$$
\begin{align}
& u_{j+1/2}^{n+1/2} = \frac{1}{2} \left (u_{j+1}^{n} + u_{j}^{n} \right )
 - \frac{\Delta t}{2 \Delta x} \left (F(u_{j+1}^{n}) - F(u_{j}^{n}) \right ) 
\tag{7}\\ 
& u_{j-1/2}^{n+1/2} = \frac{1}{2} \left (u_{j}^{n} + u_{j-1}^{n} \right )
 - \frac{\Delta t}{2 \Delta x} \left (F(u_{j}^{n}) - F(u_{j-1}^{n}) \right ) 
\end{align}
$$


$$
\begin{equation}
u_{j}^{n+1} = u_{j}^{n} - \frac{\Delta t}{\Delta x} 
\left ( F(u_{j+1/2}^{n+1/2}) - F(u_{j-1/2}^{n+1/2}) \right )
\end{equation}
$$

<p>
Это может быть преодолено:

$$
\begin{equation}
\begin{array}{c}
\dfrac{\partial u}{\partial t}\Bigg|_j^n = -\dfrac{\partial F}{\partial x}\Bigg|_j^n 
 \qquad \text{ and} \qquad
\dfrac{\partial^2u}{\partial t^2}\Bigg|_j^n = -\dfrac{\partial^2F(u)}{\partial t \partial x }\Bigg|_j^n = -\dfrac{\partial \left(\frac{\partial F(u)}{\partial t}\right)}{\partial x }\Bigg|_j^n 
\\ = -\dfrac{\partial \left(\frac{\partial F(u)}{\partial u}  \frac{\partial u}{\partial t}\right)}{\partial x }\Bigg|_j^n = 
\dfrac{\partial \left(a(u)  \frac{\partial F}{\partial x}\right)}{\partial x }\Bigg|_j^n
\end{array}
\end{equation}
$$

<p>
Теперь, подставляя в ряд Тейлора, получаем

$$
\begin{equation}
u_{j}^{n+1} = u_{j}^{n} - \Delta t \frac{\partial F(u)}{\partial x} + \frac{\Delta t^2}{2} \dfrac{\partial \left(a(u)  \frac{\partial F}{\partial x}\right)}{\partial x }
\end{equation}
$$

<p>
и далее мы можем получить общий одноступенчатый метод Лакса-Вендроффа для общего уравнения переноса.

$$
\begin{equation}
u_{j}^{n+1} = u_{j}^{n} - \frac{\Delta t}{2 \Delta x}\left(F_{j+1} - F_{j-1}\right) + \frac{\Delta t^2}{2 \Delta x^2} \Big[a_{j+1/2}\left(F_{j+1} - F_{j}\right) - a_{j-1/2}\left(F_{j} - F_{j-1}\right)\Big]
\end{equation}
$$

<p>
где $a(u)$ это скорость потока, или якобиан $F$, $F'(u)$, то есть $u$ для уравнения Бюргенса. 
Как указано, $a(u)$ следует аппроксимировать на индексах $(j+1/2)$ и $(j-1/2)$. 
Это можно сделать просто усреднением соседних значений:
$$
\begin{equation}
a_{j+1/2} = \frac{1}{2}\left(u_{j}^{n} + u_{j+1}^{n}\right)
\end{equation}
$$

для уравнения Бюргенса. Другой метод, обеспечивающий сохранность, 
    заключается в использовании следующего приближенного значения
$$
\begin{equation}
a_{j+1/2}= \left\{
        \begin{array}{ll}
            \frac{F_{j+1}^n-F_{j}^n}{u_{j+1}^n-u_{j}^n} \quad if \quad u_{j+1}\neq u_{j} \\ 
             u_{j} \quad \quad \quad otherwise
        \end{array}
    \right.
\end{equation}
$$
    
## Ссылки:
    
1. [Wiki](https://en.wikipedia.org/wiki/Lax%E2%80%93Wendroff_method)

2. [Numerical Methods for Engineers](https://folk.ntnu.no/leifh/teaching/tkt4140/._main072.html#ch:6_Lax-Wendroff_nonlin)
    
3. [Numerical Schemes Applied to theBurgers and Buckley-Leverett Equations](https://www.reading.ac.uk/web/files/maths/rakhib_ahmed.pdf)
4. [Numerical Methods for Hyperbolic Conservation Laws(AM257)](https://mathema.tician.de/dl/academic/notes/257/257.pdf)

In [3]:
def lax_wendroff_scheme(u, F, dt, dx, **kwargs):
    # scheme based on Richtmyer method
    u_nj1 = u.copy()
    ujm = u[:-2].copy()  # u(j-1)
    uj = u[1:-1].copy()  # u(j)
    ujp = u[2:].copy()  # u(j+1)
    up_m = 0.5 * (ujm + uj) - 0.5 * (dt / dx) * (F(uj) - F(ujm))  # u(n+0.5dt,j-0.5dx)
    up_p = 0.5 * (uj + ujp) - 0.5 * (dt / dx) * (F(ujp) - F(uj))  # u(n+0.5dt,j+0.5dx)

    u_nj1[1:-1] = uj - (dt / dx) * (F(up_p) - F(up_m))
    return u_nj1, u

# Метод Хартена

![](img/img1.png)

![](img/img2.png)

![](img/img3.png)

## Ссылки:

1. [High Resolution Schemes for Hyperbolic Conservation Laws](https://www.sciencedirect.com/science/article/pii/0021999183901365)

2. [СРАВНЕНИЕ ТОЧНОСТИ И СХОДИМОСТИНЕКОТОРЫХ TVD-СХЕМ](https://cyberleninka.ru/article/n/sravnenie-tochnosti-i-shodimosti-nekotoryh-tvd-shem/viewer)

3. [Numerical Methods for Hyperbolic Conservation Laws(AM257)](https://mathema.tician.de/dl/academic/notes/257/257.pdf)

In [4]:
def harten_method(u, F, dt, dx, **kwargs):
    u_nj1 = u.copy()

    ujmm = u[:-4].copy()  # u(j-2)
    ujm = u[1:-3].copy()  # u(j-1)
    uj = u[2:-2].copy()  # u(j)
    ujp = u[3:-1].copy()  # u(j+1)
    ujpp = u[4:].copy()  # u(j+2)

    a_m = a_j(uj, ujm, F)  # a(n, j-0.5)
    a_p = a_j(ujp, uj, F)  # a(n, j+0.5)

    gjmm = 0.5 * Q_func(a_m, **kwargs) * minmod(ujm - ujmm, uj - ujm)  # g_j[a(n, j-0.5)]
    gjm = 0.5 * Q_func(a_m, **kwargs) * minmod(uj - ujm, ujp - uj)  # g_j+1[a(n, j-0.5)]

    gjp = 0.5 * Q_func(a_p, **kwargs) * minmod(uj - ujm, ujp - uj)  # g_j[a(n, j+0.5)]
    gjpp = 0.5 * Q_func(a_p, **kwargs) * minmod(ujp - uj, ujpp - ujp)  # g_j+1[a(n, j+0.5)]

    gamma_jm = gamma_j_hart(gjm, gjmm, uj, ujm)  # gamma(n, j-0.5)
    gamma_jp = gamma_j_hart(gjpp, gjp, ujp, uj)  # gamma(n, j+0.5)

    fjm = 0.5 * (F(uj) + F(ujm) + 0.5 * Q_func(a_m, **kwargs) *
                 (gjm + gjmm) - Q_func(a_m + gamma_jm) * (uj - ujm))  # f(n, j-0.5)

    fjp = 0.5 * (F(ujp) + F(uj) + 0.5 * Q_func(a_p, **kwargs) *
                 (gjp + gjpp) - Q_func(a_p + gamma_jp) * (ujp - uj))  # f(n, j+0.5)

    u_nj1[2:-2] = uj - (fjp - fjm) * dt / dx
    return u_nj1, u


def minmod(x, y):
    return np.sign(x) * np.maximum(0.0, np.minimum(np.abs(x), np.sign(x) * y))


def gamma_j_hart(gjp, gj, ujp, uj):
    eq_eps = 1e-5  # machine accuracy

    def fix(x1, x2, y1, y2):
        if np.abs(y1 - y2) < eq_eps:
            return 0
        else:
            return (x1 - x2) / (y1 - y2)

    fix = np.vectorize(fix)
    return fix(gjp, gj, ujp, uj)


# Уравнение Бюргерса

Рассмотрим решение уравенения

$$
\begin{equation}
\frac{\partial u}{\partial t} + \frac{\partial F}{\partial x} = 0, \qquad F(u) = \frac{1}{2} u^2
\end{equation}
$$

In [5]:
class Burger:
    def __init__(self, N, tmax):
        self.N = N  # number of nodes
        self.tmax = tmax
        self.xmin = 0
        self.xmax = 1.5
        self.dt = 0.009  # timestep
        self.u0_type = 0

        self.init_grid()
        self.init_u()
        self.init_params()

    def init_grid(self):
        self.dx = (self.xmax - self.xmin) / self.N
        self.x = np.arange(self.xmin - self.dx, self.xmax + (2 * self.dx), self.dx)

    def init_u(self):
        u0 = np.zeros(len(self.x))
        if self.u0_type == 0:
            u0[self.x > 0.5] = 1
        elif self.u0_type == 1:
            u0[self.x < 0.5] = 1
        elif self.u0_type == 2:
            u0 = np.sin(2 * np.pi * self.x)
        elif self.u0_type == 3:
            u0[self.x <= 0.5] = - 0.5
            u0[self.x > 0.5] = 1
            u0[self.x >= 1] = 0

        self.u_nj = u0.copy()
        self.u_n1j = self.u_nj.copy()
        self.F = lambda x: 0.5 * x ** 2

    def init_params(self):
        self.nsteps = round(self.tmax / self.dt)

    def solve(self, solver, **kwargs):
        self.init_u()
        tc = 0
        frames = []  # for storing the generated images
        times = []

        for i in range(self.nsteps):
            self.u_n1j, self.u_nj = solver(self.u_n1j, self.F, self.dt, self.dx, **kwargs)

            frames.append(self.u_nj)
            times.append(tc)

            tc += self.dt

        return frames, times

    def plot_solution(self, u0_type, is_save=False):
        self.u0_type = u0_type

        frames = []
        methods = []

        fr, times = self.solve(lax_wendroff_scheme)
        methods.append(['Lax-Wendroff (Richtmyer method)'])
        frames.append([fr])

        entr_eps = [0.2]
        q_modes = [3] * len(entr_eps)
        for q, eps in zip(q_modes, entr_eps):
            fr, times = self.solve(harten_method, q_mode=q, eps=eps)
            methods.append([f'Lax-Wendroff (Harten), eps={eps})'])
            frames.append([fr])

        q_modes = [0, 1]
        entr_eps = [0.01] * len(q_modes)
        for q, eps in zip(q_modes, entr_eps):
            fr, times = self.solve(lax_friedrich_scheme, q_mode=q, eps=eps)
            methods.append([f'Lax-Friedrichs (Q_mode={q})'])
            frames.append([fr])

        q_modes = [2, 3]
        entr_eps = [0.01] * len(q_modes)
        for q, eps in zip(q_modes, entr_eps):
            fr, times = self.solve(lax_friedrich_scheme, q_mode=q, eps=eps)
            methods.append([f'Lax-Friedrichs (Q_mode={q}, eps={eps})'])
            frames.append([fr])

        frames = np.concatenate(frames)
        methods = np.concatenate(methods)

        fig, ax = plt.subplots(figsize=(6.4 * 2, 4.8 * 2))

        def update(idx):
            ax.clear()

            ax.axis((self.xmin - 0.12, self.xmax + 0.12, -1.3, 1.6))
            for k in range(len(frames)):
                ax.plot(self.x, frames[k][idx], label=methods[k])
            ax.set_title("Time = %1.3f" % (times[idx]))

            ax.grid(True)
            ax.set_xlabel("Distance (x)")
            ax.set_ylabel("$u$")
            ax.legend(loc='lower right', fontsize=12)

        fps = 30
        ani = animation.FuncAnimation(fig, update, self.nsteps, interval=1000 / fps)
        if is_save:
            writergif = animation.PillowWriter(fps=30)
            ani.save('animation_2.gif', writer=writergif)
        plt.close()

        return ani

In [6]:
burg = Burger(100, 0.6)

## Q_mode

Значение параметра ```Q_mode``` определяет тип численной вязкости:

1. ```Q_mode``` = 0: $Q(x) = 0$ **без энтропийной коррекции**
1. ```Q_mode``` = 1: $Q(x) = 1$ **без энтропийной коррекции**
1. ```Q_mode``` = 2: $Q(x) = |x|$ **с энтропийной коррекцией $\epsilon$**
1. ```Q_mode``` = 3: $Q(x) = x^2$ **с энтропийной коррекцией $\epsilon$**

Энтропийная коррекция вида:
$$
\begin{equation}
Q(x)= \left\{
        \begin{array}{ll}
            |x|, \quad \quad |x| >= \epsilon \\ 
             \frac{x^2 + \epsilon^2}{2\epsilon}, \quad \quad x < \epsilon
        \end{array}
    \right.
\end{equation}
$$

## Начальные условия вида: 
$$
\begin{equation}
u(x, 0)= \left\{
        \begin{array}{ll}
            1, \quad \quad x > 0.5 \\ 
             0, \quad \quad x < 0.5
        \end{array}
    \right.
\end{equation}
$$

In [7]:
ani = burg.plot_solution(u0_type=0)
HTML(ani.to_html5_video())

## Начальные условия вида: 
$$
\begin{equation}
u(x, 0)= \left\{
        \begin{array}{ll}
            1, \quad \quad x < 0.5 \\ 
             0, \quad \quad x > 0.5
        \end{array}
    \right.
\end{equation}
$$

In [8]:
ani = burg.plot_solution(u0_type=1)
HTML(ani.to_html5_video())

## Начальные условия вида: 
$$
\begin{equation}
u(x, 0)= sin(2\pi x)
\end{equation}
$$

In [9]:
ani = burg.plot_solution(u0_type=2)
HTML(ani.to_html5_video())

## Начальные условия вида: 
$$
\begin{equation}
u(x, 0)= \left\{
        \begin{array}{ll}
            -\frac{1}{2}, \quad \quad x < 0.5 \\ 
             1, \quad \quad x > 0.5 \\
             0, \quad \quad x >= 1 \\
        \end{array}
    \right.
\end{equation}
$$

In [10]:
ani = burg.plot_solution(u0_type=3)
HTML(ani.to_html5_video())